In [1]:
from pyspark import SparkConf, SparkContext

In [4]:
conf = SparkConf().setMaster("local").setAppName("Similarity")
sc = SparkContext(conf=conf)

# 讀取文件

In [6]:
t_all = sc.wholeTextFiles("C:\\Users\\admin\\athletics\\*")

In [7]:
t_all.take(1)

[('file:/C:/Users/admin/athletics/001.txt',
  'Claxton hunting first major medal\n\nBritish hurdler Sarah Claxton is confident she can win her first major medal at next month\'s European Indoor Championships in Madrid.\n\nThe 25-year-old has already smashed the British record over 60m hurdles twice this season, setting a new mark of 7.96 seconds to win the AAAs title. "I am quite confident," said Claxton. "But I take each race as it comes. "As long as I keep up my training but not do too much I think there is a chance of a medal." Claxton has won the national 60m hurdles title for the past three years but has struggled to translate her domestic success to the international stage. Now, the Scotland-born athlete owns the equal fifth-fastest time in the world this year. And at last week\'s Birmingham Grand Prix, Claxton left European medal favourite Russian Irina Shevchenko trailing in sixth spot.\n\nFor the first time, Claxton has only been preparing for a campaign over the hurdles - whi

# 一、將Docments轉為Shingles

## 1.去除標點符號、空格

In [157]:
topics = t_all.map(lambda x: (x[0][-7:-4],x[1].translate(str.maketrans("\n"," ",string.punctuation)).strip().replace("  "," ")))

In [9]:
import string

In [158]:
topics.take(1)

[('001',
  'Claxton hunting first major medal British hurdler Sarah Claxton is confident she can win her first major medal at next months European Indoor Championships in Madrid The 25yearold has already smashed the British record over 60m hurdles twice this season setting a new mark of 796 seconds to win the AAAs title I am quite confident said Claxton But I take each race as it comes As long as I keep up my training but not do too much I think there is a chance of a medal Claxton has won the national 60m hurdles title for the past three years but has struggled to translate her domestic success to the international stage Now the Scotlandborn athlete owns the equal fifthfastest time in the world this year And at last weeks Birmingham Grand Prix Claxton left European medal favourite Russian Irina Shevchenko trailing in sixth spot For the first time Claxton has only been preparing for a campaign over the hurdles which could explain her leap in form In previous seasons the 25yearold also 

## 2.把每個文件中字分開以便後續製作3-shingles

In [159]:
topic_split = topics.map(lambda x: (x[0],x[1].split(" ")))

In [160]:
topic_split.take(1)

[('001',
  ['Claxton',
   'hunting',
   'first',
   'major',
   'medal',
   'British',
   'hurdler',
   'Sarah',
   'Claxton',
   'is',
   'confident',
   'she',
   'can',
   'win',
   'her',
   'first',
   'major',
   'medal',
   'at',
   'next',
   'months',
   'European',
   'Indoor',
   'Championships',
   'in',
   'Madrid',
   'The',
   '25yearold',
   'has',
   'already',
   'smashed',
   'the',
   'British',
   'record',
   'over',
   '60m',
   'hurdles',
   'twice',
   'this',
   'season',
   'setting',
   'a',
   'new',
   'mark',
   'of',
   '796',
   'seconds',
   'to',
   'win',
   'the',
   'AAAs',
   'title',
   'I',
   'am',
   'quite',
   'confident',
   'said',
   'Claxton',
   'But',
   'I',
   'take',
   'each',
   'race',
   'as',
   'it',
   'comes',
   'As',
   'long',
   'as',
   'I',
   'keep',
   'up',
   'my',
   'training',
   'but',
   'not',
   'do',
   'too',
   'much',
   'I',
   'think',
   'there',
   'is',
   'a',
   'chance',
   'of',
   'a',
   'meda

## 3. 建立一個shingle set

In [25]:
sh_set = []

In [161]:
# 每三個字為單位，每次向後一個字，append到shingle set
for c in topic_split.collect()[1]:
    for i in range(0,len(c)-2):
        sh_set.append(c[i:i+3])

## 4.讓每個文件跟shingle set作比對

In [162]:
#製作出每個文件自己部分的shingle
def self_set(x):
    splited = x[1].split(" ")
    self_set = []
    for i in range(0,len(splited)-2):
        self_set.append(splited[i:i+3])
    return (x[0],self_set)

In [164]:
topics_self = topics.map(self_set)

In [165]:
topics_self.take(2)

[('001',
  [['Claxton', 'hunting', 'first'],
   ['hunting', 'first', 'major'],
   ['first', 'major', 'medal'],
   ['major', 'medal', 'British'],
   ['medal', 'British', 'hurdler'],
   ['British', 'hurdler', 'Sarah'],
   ['hurdler', 'Sarah', 'Claxton'],
   ['Sarah', 'Claxton', 'is'],
   ['Claxton', 'is', 'confident'],
   ['is', 'confident', 'she'],
   ['confident', 'she', 'can'],
   ['she', 'can', 'win'],
   ['can', 'win', 'her'],
   ['win', 'her', 'first'],
   ['her', 'first', 'major'],
   ['first', 'major', 'medal'],
   ['major', 'medal', 'at'],
   ['medal', 'at', 'next'],
   ['at', 'next', 'months'],
   ['next', 'months', 'European'],
   ['months', 'European', 'Indoor'],
   ['European', 'Indoor', 'Championships'],
   ['Indoor', 'Championships', 'in'],
   ['Championships', 'in', 'Madrid'],
   ['in', 'Madrid', 'The'],
   ['Madrid', 'The', '25yearold'],
   ['The', '25yearold', 'has'],
   ['25yearold', 'has', 'already'],
   ['has', 'already', 'smashed'],
   ['already', 'smashed', 'the'],

In [163]:
#和所有文件的shingle作比對(拿每個文件自己的shingle set和所有文件製成的shingle set比對)
def shingles(x):
    shingle = []
    for i in sh_set:
        if i in x[1]:
            shingle.append("1")
        else:
            shingle.append("0")
    return (x[0],shingle)

In [166]:
topics_shingles = topics_self.map(shingles)

In [167]:
topics_shingles.take(2)

[('001',
  ['1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',
   '1',

# 二、MinHash

## 1.做出100個不同的hash functions

In [47]:
import numpy as np

In [373]:
#h = ((a*x)+b) % c is the standard, accepted way to generate hash functions.
#Also, a and b should be random numbers from the range 1 to c-1

In [374]:
#為了確保100的都是不同的function->所有(a,b)的集合裡的個數為100個

In [183]:
hash_list = [(np.random.randint(low=1, high=3406),np.random.randint(low=1, high=3406)) for i in range(200)]

In [184]:
hash_set = set(hash_list)

In [185]:
#製作了200個
len(hash_set)

200

In [186]:
valid_hash = list(hash_set)

In [187]:
#留100個
valid_hash = valid_hash[:100]

In [176]:
valid_hash

[(7, 3),
 (11, 11),
 (14, 4),
 (13, 4),
 (12, 12),
 (1, 6),
 (5, 8),
 (6, 7),
 (5, 5),
 (11, 5),
 (10, 7),
 (7, 6),
 (6, 10),
 (14, 1),
 (13, 7),
 (1, 1),
 (4, 10),
 (3, 2),
 (2, 6),
 (9, 14),
 (8, 2),
 (5, 11),
 (4, 5),
 (10, 13),
 (9, 3),
 (7, 5),
 (12, 11),
 (14, 2),
 (3, 12),
 (8, 12),
 (3, 1),
 (9, 9),
 (5, 14),
 (6, 13),
 (14, 8),
 (12, 8),
 (13, 13),
 (3, 11),
 (2, 1),
 (8, 9),
 (4, 12),
 (9, 4),
 (10, 3),
 (7, 2),
 (6, 14),
 (12, 2),
 (13, 3),
 (3, 6),
 (1, 10),
 (8, 6),
 (10, 9),
 (6, 4),
 (5, 4),
 (6, 11),
 (12, 7),
 (11, 9),
 (14, 6),
 (13, 6),
 (3, 5),
 (9, 13),
 (8, 3),
 (5, 10),
 (4, 6),
 (10, 10),
 (9, 2),
 (6, 1),
 (5, 7),
 (12, 4),
 (14, 3),
 (13, 9),
 (1, 3),
 (8, 13),
 (9, 8),
 (5, 13),
 (11, 14),
 (7, 11),
 (14, 9),
 (12, 9),
 (3, 10),
 (1, 14),
 (8, 10),
 (4, 13),
 (2, 13),
 (11, 13),
 (7, 14),
 (14, 10),
 (13, 2),
 (12, 14),
 (1, 9),
 (9, 6),
 (6, 5),
 (6, 8),
 (11, 8),
 (7, 13),
 (14, 7),
 (13, 5),
 (3, 4),
 (2, 4),
 (9, 12),
 (4, 7)]

In [375]:
#定義100個hash function (c = 3407)

In [188]:
for i in range(100):
    def h(i,x):
        return ((valid_hash[i][0]*x)+valid_hash[i][1])%3407

In [ ]:
#算出每個文件在100個hash function下hash100次的100個hash value

## 2. minhash製作出每個文件的signature

In [189]:
def minhashing(x):
    signatures = []
    #一次一個hash function
    for i in range(100):
        min_hash = 200
        for c in range(len(x[1])):  #len(x)>30000
            #只有在shingle是"1"的時候，才有hash value
            if x[1][c] == "1":
                # 每次都update最小的min hash value 
                if min_hash > h(i,c):
                    min_hash = h(i,c)
        signatures.append(min_hash)
    return (x[0],signatures)

In [190]:
minhash = topics_shingles.map(minhashing)

In [249]:
minhash.take(1)

[('001',
  [4,
   7,
   18,
   12,
   7,
   13,
   17,
   4,
   11,
   3,
   1,
   6,
   50,
   0,
   3,
   2,
   0,
   3,
   3,
   5,
   9,
   4,
   5,
   5,
   48,
   12,
   44,
   9,
   8,
   1,
   8,
   3,
   0,
   12,
   9,
   8,
   17,
   4,
   7,
   16,
   5,
   0,
   12,
   10,
   1,
   0,
   15,
   6,
   13,
   0,
   4,
   1,
   5,
   1,
   7,
   4,
   39,
   8,
   13,
   11,
   3,
   61,
   2,
   27,
   4,
   6,
   13,
   31,
   20,
   0,
   11,
   4,
   1,
   2,
   5,
   7,
   1,
   7,
   6,
   0,
   5,
   9,
   11,
   2,
   30,
   0,
   8,
   17,
   0,
   1,
   3,
   18,
   2,
   18,
   14,
   28,
   8,
   1,
   2,
   6])]

# 三、LSH(b=50,r=2)

## 1.將100個siganature2個2個一組分成50組

In [192]:
#將100個siganature2個2個分成50組
def cut_bands(x):
    bands = []
    for i in range(50):
        bands.append((x[1][i*2],x[1][i*2+1]))
    return (x[0],bands)

In [193]:
bands = minhash.map(cut_bands)

In [194]:
bands.take(1)

[('001',
  [(4, 7),
   (18, 12),
   (7, 13),
   (17, 4),
   (11, 3),
   (1, 6),
   (50, 0),
   (3, 2),
   (0, 3),
   (3, 5),
   (9, 4),
   (5, 5),
   (48, 12),
   (44, 9),
   (8, 1),
   (8, 3),
   (0, 12),
   (9, 8),
   (17, 4),
   (7, 16),
   (5, 0),
   (12, 10),
   (1, 0),
   (15, 6),
   (13, 0),
   (4, 1),
   (5, 1),
   (7, 4),
   (39, 8),
   (13, 11),
   (3, 61),
   (2, 27),
   (4, 6),
   (13, 31),
   (20, 0),
   (11, 4),
   (1, 2),
   (5, 7),
   (1, 7),
   (6, 0),
   (5, 9),
   (11, 2),
   (30, 0),
   (8, 17),
   (0, 1),
   (3, 18),
   (2, 18),
   (14, 28),
   (8, 1),
   (2, 6)])]

In [83]:
#hash to the same bucket for ≧ 1 band

## 2.製作出每個文件的50組分別被hash到哪一個bucket

In [195]:
# 製作一個hash function(這裡我讓每一組(a,b)，(a*23+b)%51)
def hash_fun(x):
    bucket_ord = []
    for i in x[1]:
        bucket_ord.append((i[0]*23+i[1])%51)
    return (x[0],bucket_ord)

In [196]:
#算出每個(a,b)分別被hash到哪個bucket
bands_hash = bands.map(hash_fun)

In [273]:
bands_hash.take(1)

[('001',
  [48,
   18,
   21,
   38,
   1,
   29,
   28,
   20,
   3,
   23,
   7,
   18,
   45,
   1,
   32,
   34,
   12,
   11,
   38,
   24,
   13,
   31,
   23,
   45,
   44,
   42,
   14,
   12,
   38,
   4,
   28,
   22,
   47,
   24,
   1,
   2,
   25,
   20,
   30,
   36,
   22,
   0,
   27,
   48,
   1,
   36,
   13,
   44,
   32,
   1])]

# 三、計算相似度

邏輯如下:
+ 1.紀錄所有文件的(a,b)->屬於哪一個文件、哪一個band(a)、被hash到哪個bucket(b)->(band的index(a),文件編號，所屬bucket的index(b))
+ 2.每個band自己做比較: 以band index為key，比較不同文件所屬的bucket->如果bucket的index代表這兩個文件在這個band被認為是相似的
+ 3.把每個band認為相似的文件pair加起來(ex.如果對於所有band來說(d1,d2)都是相似的->加起來會是50)就會是兩個文件的相似程度
+ 4.算similarity->把加出來的值除以50

## 1.紀錄所有文件的(a,b)

In [210]:
#屬於哪一個文件、哪一個band、被hash到哪個bucket->(band的index,文件編號，所屬bucket的index)
def index_doc_value(x):
    tripple_list = []
    for i in range(len(x[1])):
        tripple_list.append((i,(x[0],x[1][i])))
    return tripple_list

In [211]:
bands_filter = bands_hash.flatMap(index_doc_value)

In [212]:
bands_filter.take(2)

[(0, ('001', 48)), (1, ('001', 18))]

## 2.每個band自己做比較

In [215]:
#1.以band index為key分群
candidate_filter = bands_filter.groupByKey().map(lambda x: (x[0],list(x[1])))

In [277]:
candidate_filter.take(1)

[(0,
  [('001', 48),
   ('002', 45),
   ('003', 24),
   ('004', 11),
   ('005', 49),
   ('006', 36),
   ('007', 50),
   ('008', 5),
   ('009', 14),
   ('010', 27),
   ('011', 17),
   ('012', 27),
   ('013', 18),
   ('014', 16),
   ('015', 8),
   ('016', 48),
   ('017', 48),
   ('018', 18),
   ('019', 49),
   ('020', 27),
   ('021', 0),
   ('022', 41),
   ('023', 46),
   ('024', 22),
   ('025', 36),
   ('026', 44),
   ('027', 37),
   ('028', 44),
   ('029', 11),
   ('030', 46),
   ('031', 19),
   ('032', 36),
   ('033', 1),
   ('034', 9),
   ('035', 46),
   ('036', 9),
   ('037', 24),
   ('038', 48),
   ('039', 35),
   ('040', 1),
   ('041', 50),
   ('042', 36),
   ('043', 15),
   ('044', 24),
   ('045', 24),
   ('046', 32),
   ('047', 0),
   ('048', 18),
   ('049', 0),
   ('050', 29),
   ('051', 46),
   ('052', 19),
   ('053', 26),
   ('054', 2),
   ('055', 41),
   ('056', 43),
   ('057', 1),
   ('058', 23),
   ('059', 44),
   ('060', 24),
   ('061', 25),
   ('062', 5),
   ('063', 36),

In [349]:
# 2.比較不同文件所屬的bucket->如果bucket的index代表這兩個文件在這個band被認為是相似的
def pair_collection(x):
    pair_collection = []
    #同一個band
    #不同文件屬於這個band的部分作比較
    for i in range(len(x[1])):
        for c in range(len(x[1])):
            if i >= c:
                continue
            #同一個bucket -> 在這個band被認為是相似的
            elif x[1][i][1] == x[1][c][1]:
                pair_collection.append((x[1][i][0],x[1][c][0]))
    return pair_collection

## 3.把每個band認為相似的文件pair加起來就會是兩個文件的相似程度
+ ex.如果對於所有band來說(d1,d2)都是相似的->加起來會是50

### (1)method 1

In [369]:
# 用Counter在計算在每個band裡面的相似pair的個數(不管是哪一個candidate pair在同一個band裡的Counter都只會是1)
row_count = candidate_filter.map(pair_collection).map(lambda x: Counter(x))

In [370]:
# 把每一個band的Counter加起來
all_count = a.reduce(lambda x,y: x+y)

In [371]:
all_count

Counter({('001', '016'): 2,
         ('001', '017'): 1,
         ('001', '038'): 2,
         ('003', '037'): 15,
         ('003', '044'): 1,
         ('003', '045'): 8,
         ('003', '060'): 6,
         ('003', '077'): 2,
         ('003', '091'): 6,
         ('004', '029'): 3,
         ('004', '083'): 3,
         ('004', '100'): 1,
         ('005', '019'): 5,
         ('005', '065'): 1,
         ('005', '080'): 8,
         ('006', '025'): 2,
         ('006', '032'): 3,
         ('006', '042'): 2,
         ('006', '063'): 2,
         ('006', '067'): 2,
         ('006', '099'): 1,
         ('007', '041'): 1,
         ('008', '062'): 7,
         ('008', '082'): 5,
         ('009', '086'): 1,
         ('010', '012'): 1,
         ('010', '020'): 1,
         ('011', '078'): 2,
         ('012', '020'): 50,
         ('013', '018'): 1,
         ('013', '048'): 1,
         ('013', '076'): 1,
         ('015', '074'): 1,
         ('015', '087'): 2,
         ('016', '017'): 2,
         ('016', '

### (2)method 2

In [372]:
# 用Flatmap把每一個band的candidate pair放在同一地方(我們不在乎candidate pair是哪一個band，只在乎個數)
candidate_filter1 = candidate_filter.map(pair_collection).flatMap(lambda x: x)

In [307]:
candidate_filter1.take(1)

[('001', '016')]

In [303]:
resume = list(candidate_filter1.collect())

In [312]:
# 算出candidate pair分別被多少個band認為是相似的
Count = Counter(resume)

In [313]:
Count

Counter({('001', '016'): 2,
         ('001', '017'): 1,
         ('001', '038'): 2,
         ('003', '037'): 15,
         ('003', '044'): 1,
         ('003', '045'): 8,
         ('003', '060'): 6,
         ('003', '077'): 2,
         ('003', '091'): 6,
         ('004', '029'): 3,
         ('004', '083'): 3,
         ('004', '100'): 1,
         ('005', '019'): 5,
         ('005', '065'): 1,
         ('005', '080'): 8,
         ('006', '025'): 2,
         ('006', '032'): 3,
         ('006', '042'): 2,
         ('006', '063'): 2,
         ('006', '067'): 2,
         ('006', '099'): 1,
         ('007', '041'): 1,
         ('008', '062'): 7,
         ('008', '082'): 5,
         ('009', '086'): 1,
         ('010', '012'): 1,
         ('010', '020'): 1,
         ('011', '078'): 2,
         ('012', '020'): 50,
         ('013', '018'): 1,
         ('013', '048'): 1,
         ('013', '076'): 1,
         ('015', '074'): 1,
         ('015', '087'): 2,
         ('016', '017'): 2,
         ('016', '

## 4.算出Similarity

In [331]:
# 去除掉被少於2個band認為是相似的pair(也就是只有一個band認為他們是相似的)->candidate pair
Ultimate_candidate = dict([(k,v) for k, v in filter(lambda x: x[1]>=2, list(Count.items()))])

In [333]:
# 將每個candidate pair被認為是相似的數量 除以 50 就是 Similarity
Similarity = [(k,v/50) for k,v in Ultimate_candidate.items()]

In [334]:
Similarity

[(('001', '016'), 0.04),
 (('001', '038'), 0.04),
 (('003', '037'), 0.3),
 (('003', '045'), 0.16),
 (('003', '060'), 0.12),
 (('003', '077'), 0.04),
 (('003', '091'), 0.12),
 (('004', '029'), 0.06),
 (('004', '083'), 0.06),
 (('005', '019'), 0.1),
 (('005', '080'), 0.16),
 (('006', '025'), 0.04),
 (('006', '032'), 0.06),
 (('006', '042'), 0.04),
 (('006', '063'), 0.04),
 (('006', '067'), 0.04),
 (('008', '062'), 0.14),
 (('008', '082'), 0.1),
 (('011', '078'), 0.04),
 (('012', '020'), 1.0),
 (('015', '087'), 0.04),
 (('016', '017'), 0.04),
 (('016', '038'), 0.04),
 (('019', '065'), 0.06),
 (('019', '080'), 0.06),
 (('021', '049'), 0.04),
 (('021', '088'), 0.08),
 (('023', '030'), 0.14),
 (('023', '035'), 0.16),
 (('023', '051'), 0.08),
 (('024', '072'), 0.08),
 (('025', '042'), 0.28),
 (('025', '063'), 0.06),
 (('025', '067'), 0.04),
 (('025', '099'), 0.14),
 (('026', '028'), 0.04),
 (('026', '059'), 0.04),
 (('028', '059'), 0.14),
 (('028', '090'), 0.38),
 (('029', '100'), 0.04),
 (('

# 四、Top 10

In [344]:
Top_10 = sorted(Similarity,key = lambda x: x[1],reverse = True)[:10]

In [345]:
Top_10

[(('012', '020'), 1.0),
 (('052', '084'), 1.0),
 (('047', '049'), 0.76),
 (('030', '035'), 0.58),
 (('045', '091'), 0.46),
 (('061', '069'), 0.4),
 (('040', '049'), 0.4),
 (('028', '090'), 0.38),
 (('027', '101'), 0.38),
 (('048', '049'), 0.36)]